In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from tic_env import TictactoeEnv, OptimalPlayer

# Tic Toc Toe environment

Our 1st game is the famous Tic Toc Toe. You can read about the game and its rules here: https://en.wikipedia.org/wiki/Tic-tac-toe

We implemented the game as an environment in the style of games in the [Python GYM library](https://gym.openai.com/). The commented source code is available in the file "tic_env.py". Here, we give a brief introduction to the environment and how it can be used.

### Initialization and attributes

You can initialize the environment / game as following:

In [ ]:
env = TictactoeEnv()

Which then has the following attributes with the corresponding initial values:

In [ ]:
env.__dict__

The game is played by two players: player 'X' and player 'O'. The attribute 'current_player' shows whose turn it is. We assume that player 'X' always plays first.

The attribute 'grid' is a 3x3 numpy array and presents the board in the real game and the state $s_t$ in the reinfocement learning language. Each elements can take a value in {0, 1, -1}:
     0 : place unmarked
     1 : place marked with X 
    -1 : place marked with O 
        
The attribute 'end' shows if the game is over or not, and the attribute 'winner' shows the winner of the game: either "X", "O", or None.  

You can use function 'render' to visualize the current position of the board:

In [ ]:
env.render()

### Taking actions

The game environment will recieve action from two players in turn and update the grid. At each time, one player can take the action $a_t$, where $a_t$ can either be an integer between 0 to 8 or a touple, corresponding to the 9 possible.

Function 'step' is used to recieve the action of the player, update the grid:

In [ ]:
env.step(2)

In [ ]:
env.render()

In [ ]:
env.__dict__

In [ ]:
env.step((1,1))

In [ ]:
env.render()

In [ ]:
env.__dict__

But not all actions are available at each time: One cannot choose a place which has been taken before. There is an error if an unavailable action is taken:

In [ ]:
env.step((0,2))

### Reward

Reward is always 0 until the end of the game. When the game is over, the reward is 1 if you win the game, -1 if you lose, and 0 besides. Function 'observe' can be used after each step to recieve the new state $s_t$, whether the game is over, and the winner, and function 'reward' to get the reward value $r_t$:

In [ ]:
env.observe()

In [ ]:
env.reward(player='X')

In [ ]:
env.reward(player='O')

An example of finishing the game:

In [ ]:
env.step(0)
env.step(3)
env.step(1)

In [ ]:
env.render()

In [ ]:
env.observe()

In [ ]:
env.reward(player='X')

In [ ]:
env.reward(player='O')

# Optimal policy for Tic Toc Toe environment

Fortunately, we know the exact optimal policy for Tic Toc Toe. We have implemented and $\epsilon$-greedy version of optimal polciy which you can use for the project.

In [ ]:
env.reset();

In [ ]:
opt_player = OptimalPlayer(epsilon = 0., player = 'X')

In [ ]:
opt_player.act(env.grid)

In [ ]:
opt_player.player

### An example of optimal player playing against random player

In [ ]:
Turns = np.array(['X','O'])
for i in range(5):
    env.reset()
    grid, _, __ = env.observe()
    Turns = Turns[np.random.permutation(2)]
    player_opt = OptimalPlayer(epsilon=0., player=Turns[0])
    player_rnd = OptimalPlayer(epsilon=1., player=Turns[1])
    for j in range(9):
        if env.current_player == player_opt.player:
            move = player_opt.act(grid)
        else:
            move = player_rnd.act(grid)

        grid, end, winner = env.step(move, print_grid=False)

        if end:
            print('-------------------------------------------')
            print('Game end, winner is player ' + str(winner))
            print('Optimal player = ' +  Turns[0])
            print('Random player = ' +  Turns[1])
            env.render()
            env.reset()
            break


### An example of optimal player playing against optimal player

In [ ]:
Turns = np.array(['X','O'])
for i in range(5):
    env.reset()
    grid, _, __ = env.observe()
    Turns = Turns[np.random.permutation(2)]
    player_opt_1 = OptimalPlayer(epsilon=0., player=Turns[0])
    player_opt_2 = OptimalPlayer(epsilon=0., player=Turns[1])
    for j in range(9):
        if env.current_player == player_opt.player:
            move = player_opt_1.act(grid)
        else:
            move = player_opt_2.act(grid)

        grid, end, winner = env.step(move, print_grid=False)

        if end:
            print('-------------------------------------------')
            print('Game end, winner is player ' + str(winner))
            print('Optimal player 1 = ' +  Turns[0])
            print('Optimal player 2 = ' +  Turns[1])
            env.render()
            env.reset()
            break


In [ ]:
from utils import play
from tdlearner import TDPlayer
from tic_env import OptimalPlayer

random_player = OptimalPlayer(epsilon=1.0)
q_player = TDPlayer(epsilon=0.01)

play(random_player, q_player, episodes=20000, debug=False)

In [5]:
from utils import play
from deepqlearner import DeepQPlayer
from tic_env import OptimalPlayer

random_player = OptimalPlayer(epsilon=0.5)
q_player = DeepQPlayer(epsilon=0.01, target_update=500, batch_size=64)

play(random_player, q_player, episodes=20000, debug=False)

/Users/mismayil/Desktop/EPFL/W2022/ANN/ann-miniproject/deepqlearner.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.memory.append(Transition(state=torch.tensor(state),
/Users/mismayil/Desktop/EPFL/W2022/ANN/ann-miniproject/deepqlearner.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  next_state=torch.tensor(next_state) if next_state is not None else None,
/Users/mismayil/opt/anaconda3/envs/annproject/lib/python3.8/site-packages/torch/nn/modules/loss.py:981: UserWarning: Using a target size (torch.Size([64, 1, 64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.

num_games=500, loss=0.147556334733963
num_games=500, loss=0.15135550498962402
num_games=500, loss=0.19541756808757782
num_games=1000, loss=0.1475965827703476
num_games=1000, loss=0.14182719588279724
num_games=1000, loss=0.16059863567352295
num_games=1500, loss=0.1318572461605072
num_games=1500, loss=0.17981356382369995
num_games=1500, loss=0.1595131903886795
num_games=2000, loss=0.17843395471572876
num_games=2000, loss=0.18424002826213837
num_games=2000, loss=0.2033759355545044
num_games=2000, loss=0.20945656299591064
num_games=2500, loss=0.2144092321395874
num_games=2500, loss=0.20564110577106476
num_games=2500, loss=0.1858583688735962
num_games=2500, loss=0.2046937197446823
num_games=3000, loss=0.16696423292160034
num_games=3000, loss=0.24624022841453552
num_games=3000, loss=0.21909379959106445
num_games=3500, loss=0.23054607212543488
num_games=3500, loss=0.19797641038894653
num_games=3500, loss=0.21453377604484558
num_games=4000, loss=0.18760257959365845
num_games=4000, loss=0.21367

({'wins': 18881, 'losses': 1070, 'M': 0.89055},
 {'wins': 1070, 'losses': 18881, 'M': -0.89055})